In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
tf.config.run_functions_eagerly(True)

from tensorflow.keras.utils import image_dataset_from_directory

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import regnet

# Initialize rng
rng = np.random.default_rng(2022)

auc = tf.keras.metrics.AUC()

In [ ]:
# # plot diagnostic learning curves
# def summarize_diagnostics(history):
#   fig, ax = plt.subplots(1,3, figsize=(30, 10))
#   # plot loss
#   ax[0].set_title('Loss Curves', fontsize=20)
#   ax[0].plot(history.history['loss'], label='train')
#   ax[0].plot(history.history['val_loss'], label='val')
#   ax[0].set_xlabel('Epochs', fontsize=15)
#   ax[0].set_ylabel('Loss', fontsize=15)
#   ax[0].legend(fontsize=15)
#   # plot AUC
#   ax[1].set_title('Classification AUC', fontsize=20)
#   ax[1].plot(history.history['auc'], label='train')
#   ax[1].plot(history.history['val_auc'], label='val')
#   ax[1].set_xlabel('Epochs', fontsize=15)
#   ax[1].set_ylabel('AUROC', fontsize=15)
#   ax[1].legend(fontsize=15)
#   # plot accuracy
#   ax[2].set_title('Classification accuracy', fontsize=20)
#   ax[2].plot(history.history['accuracy'], label='train')
#   ax[2].plot(history.history['val_accuracy'], label='val')
#   ax[2].set_xlabel('Epochs', fontsize=15)
#   ax[2].set_ylabel('Accuracy', fontsize=15)
#   ax[2].legend(fontsize=15)

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
batch_size = 32 # This is a tunable hyperparameter
shape = (128, 128) # note we are reducing the size of the image
# Note: you will use 'grayscale' images for your own model
# but you might need to switch to 'rgb' for pretrained models because they are trained on ImageNet which has only RGB images
data_dir = '/content/drive/MyDrive/Final_Project/Dataset'
train_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'train/train'),
                                                       seed=rng.integers(500000),
                                                       image_size=shape,
                                                       label_mode="categorical",
                                                       color_mode='rgb',
                                                       batch_size=batch_size)
train_ds_new = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'train_new'),
                                                       seed=rng.integers(500000),
                                                       image_size=shape,
                                                       label_mode="categorical",
                                                       color_mode='rgb',
                                                       batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'validation/validation'),
                                                     seed=rng.integers(500000),
                                                     image_size=shape,
                                                     label_mode="categorical",
                                                     color_mode='rgb',
                                                     batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(data_dir, 'test_new'),
                                                      seed=rng.integers(500000),
                                                      image_size=shape,
                                                      label_mode="categorical",
                                                      color_mode='rgb',
                                                      batch_size=batch_size)

Found 20000 files belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Found 16000 files belonging to 4 classes.
Found 4000 files belonging to 4 classes.
Found 4000 files belonging to 4 classes.


In [ ]:
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(train_ds.class_names[labels[i]])
#     plt.axis("off")

In [ ]:
for x,y in train_ds.take(1):
  print(x.shape)
  print(y.shape)

(32, 128, 128, 3)
(32, 4)


In [23]:
#CNN with VGG19
base_model = tf.keras.applications.vgg19.VGG19(weights='imagenet',  # Load weights pre-trained on ImageNet.
                                               input_shape=(128, 128, 3), # Input shape of our images
                                               include_top=False)  # Do not include the ImageNet classifier at the top.

base_model.trainable = True
inputs = Input(shape=(128, 128, 3))
# base_model is running in inference mode here, i.e., extracting features using the trained model.
layer = base_model(inputs, training=True)
# Use a flatten layer
layer = Flatten()(layer)
# Add dropout to reduce overfitting
layer = Dropout(0.2)(layer)
# A Dense classifier with a single unit (binary classification)
outputs = Dense(4, activation='softmax')(layer)
model = Model(inputs, outputs)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dropout_3 (Dropout)         (None, 8192)              0         
                                                                 
 dense_3 (Dense)             (None, 4)                 32772     
                                                                 
Total params: 20057156 (76.51 MB)
Trainable params: 20057156 (76.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# early stopping
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=10,
                   restore_best_weights=True)

# define optimizer
opt = Adam(learning_rate=0.0005)

# Compile metrics=['acc', f1_m, precision_m, recall_m])
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', auc])
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', f1_m, precision_m, recall_m])


In [25]:
history = model.fit(train_ds_new,
                    epochs=50,
                    batch_size=batch_size,
                    validation_data=val_ds,
                    callbacks=[es],
                    verbose=1)

Epoch 1/50
500/500 [==============================] - 125s 250ms/step - loss: 1.5418 - acc: 0.2488 - f1_m: 0.0030 - precision_m: 0.0052 - recall_m: 0.0026 - val_loss: 1.3863 - val_acc: 0.2500 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/50
500/500 [==============================] - 125s 249ms/step - loss: 1.3864 - acc: 0.2437 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.3863 - val_acc: 0.2500 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/50
500/500 [==============================] - 125s 249ms/step - loss: 1.3864 - acc: 0.2429 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.3863 - val_acc: 0.2500 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/50
500/500 [==============================] - 125s 249ms/step - loss: 1.3864 - acc: 0.2441 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.00

KeyboardInterrupt: ignored

In [ ]:
# Loss and accuracy curves
# summarize_diagnostics(history)

In [ ]:
# # Evaluate model, wont work as we have no test data.
# _, test_acc, test_auc = model.evaluate(test_ds, verbose=1)
# print('AUC:', test_auc)
# print('Accuracy:', test_acc)

# Evaluate model, wont work as we have no test data.
# _, test_acc, test_auc = model.evaluate(test_ds, verbose=1)
loss, accuracy, f1_score, precision, recall = model.evaluate(test_ds, verbose=1)
# print('AUC:', test_auc)
# print('Accuracy:', test_acc)
print('loss:', loss)
print('Accuracy:', accuracy)
print('f1_score:', f1_score)
print('precision:', precision)
print('recall:', recall)

In [ ]:
#CNN with InceptionV3
base_model_2 = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet',  # Load weights pre-trained on ImageNet.
                                               input_shape=(128, 128, 3), # Input shape of our images
                                               include_top=False)  # Do not include the ImageNet classifier at the top.

base_model_2.trainable = True
inputs = Input(shape=(128, 128, 3))
# base_model is running in inference mode here, i.e., extracting features using the trained model.
layer = base_model_2(inputs, training=True)
# Use a flatten layer
layer = Flatten()(layer)
# Add dropout to reduce overfitting
layer = Dropout(0.2)(layer)
# A Dense classifier with a multiple unit (categorical classification)
outputs = Dense(4, activation='softmax')(layer)
model_2 = Model(inputs, outputs)

model_2.summary()

87910968/87910968 [==============================] - 3s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 2, 2, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dropout_1 (Dropout)         (None, 8192)              0         
                                                                 
 dense_1 (Dense)             (None, 4)                 32772     
                                                                 
Total params: 21835556 (83.30 MB)
Trainable params: 21801124 (83.16 MB)
Non-trainable params: 34432 (134.50 KB)
______________

In [ ]:
# early stopping
es_2 = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=10,
                   restore_best_weights=True)

# define optimizer
opt = Adam(learning_rate=0.0005)

# Compile metrics=['acc', f1_m, precision_m, recall_m])
# model_2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', auc])
model_2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', f1_m, precision_m, recall_m])



In [15]:
history_2 = model_2.fit(train_ds_new,
                    epochs=50,
                    batch_size=batch_size,
                    validation_data=val_ds,
                    callbacks=[es_2],
                    verbose=1)

Epoch 1/50
500/500 [==============================] - 380s 755ms/step - loss: 0.4967 - acc: 0.8257 - f1_m: 0.8189 - precision_m: 0.8486 - recall_m: 0.7948 - val_loss: 0.2862 - val_acc: 0.9022 - val_f1_m: 0.9027 - val_precision_m: 0.9108 - val_recall_m: 0.8950
Epoch 2/50
500/500 [==============================] - 382s 763ms/step - loss: 0.2689 - acc: 0.9087 - f1_m: 0.9084 - precision_m: 0.9161 - recall_m: 0.9011 - val_loss: 0.3655 - val_acc: 0.8907 - val_f1_m: 0.8886 - val_precision_m: 0.8958 - val_recall_m: 0.8817
Epoch 3/50
500/500 [==============================] - 379s 757ms/step - loss: 0.2107 - acc: 0.9281 - f1_m: 0.9282 - precision_m: 0.9330 - recall_m: 0.9237 - val_loss: 0.2756 - val_acc: 0.9053 - val_f1_m: 0.9050 - val_precision_m: 0.9099 - val_recall_m: 0.9003
Epoch 4/50
500/500 [==============================] - 379s 757ms/step - loss: 0.1552 - acc: 0.9481 - f1_m: 0.9485 - precision_m: 0.9511 - recall_m: 0.9459 - val_loss: 0.2337 - val_acc: 0.9215 - val_f1_m: 0.9219 - val_pre

In [ ]:
# Loss and accuracy curves
# summarize_diagnostics(history_2)

In [16]:
# # Evaluate model, wont work as we have no test data.
# _, test_acc, test_auc = model_2.evaluate(test_ds, verbose=1)
# print('AUC:', test_auc)
# print('Accuracy:', test_acc)

# Evaluate model, wont work as we have no test data.
# _, test_acc, test_auc = model.evaluate(test_ds, verbose=1)
loss, accuracy, f1_score, precision, recall = model_2.evaluate(test_ds, verbose=1)
# print('AUC:', test_auc)
# print('Accuracy:', test_acc)
print('loss:', loss)
print('Accuracy:', accuracy)
print('f1_score:', f1_score)
print('precision:', precision)
print('recall:', recall)

125/125 [==============================] - 27s 214ms/step - loss: 0.2399 - acc: 0.9252 - f1_m: 0.9253 - precision_m: 0.9279 - recall_m: 0.9227
loss: 0.23993845283985138
Accuracy: 0.9252499938011169
f1_score: 0.9252724051475525
precision: 0.9279080033302307
recall: 0.9227499961853027


In [17]:
#CNN with Resnet50
base_model_3 = tf.keras.applications.resnet50.ResNet50(weights='imagenet',  # Load weights pre-trained on ImageNet.
                                               input_shape=(128, 128, 3), # Input shape of our images
                                               include_top=False)  # Do not include the ImageNet classifier at the top.

base_model_3.trainable = True
inputs = Input(shape=(128, 128, 3))
# base_model is running in inference mode here, i.e., extracting features using the trained model.
layer = base_model_3(inputs, training=True)
# Use a flatten layer
layer = Flatten()(layer)
# Add dropout to reduce overfitting
layer = Dropout(0.2)(layer)
# A Dense classifier with a multiple unit (categorical classification)
outputs = Dense(4, activation='softmax')(layer)
model_3 = Model(inputs, outputs)

model_3.summary()

94765736/94765736 [==============================] - 3s 0us/step
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 32768)             0         
                                                                 
 dropout_2 (Dropout)         (None, 32768)             0         
                                                                 
 dense_2 (Dense)             (None, 4)                 131076    
                                                                 
Total params: 23718788 (90.48 MB)
Trainable params: 23665668 (90.28 MB)
Non-trainable params: 53120 (207.50 KB)
______________

In [18]:
# early stopping
es_3 = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=10,
                   restore_best_weights=True)

# define optimizer
opt = Adam(learning_rate=0.0005)

# Compile metrics=['acc', f1_m, precision_m, recall_m])
# model_3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', auc])
model_3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc', f1_m, precision_m, recall_m])



In [ ]:
history_3 = model_3.fit(train_ds_new,
                    epochs=50,
                    batch_size=batch_size,
                    validation_data=val_ds,
                    callbacks=[es_3],
                    verbose=1)

Epoch 1/50
500/500 [==============================] - 343s 686ms/step - loss: 0.1345 - acc: 0.9529 - f1_m: 0.9532 - precision_m: 0.9556 - recall_m: 0.9510 - val_loss: 0.3112 - val_acc: 0.9170 - val_f1_m: 0.9166 - val_precision_m: 0.9189 - val_recall_m: 0.9145
Epoch 2/50
500/500 [==============================] - 343s 686ms/step - loss: 0.1213 - acc: 0.9598 - f1_m: 0.9602 - precision_m: 0.9621 - recall_m: 0.9584 - val_loss: 0.3166 - val_acc: 0.9085 - val_f1_m: 0.9098 - val_precision_m: 0.9118 - val_recall_m: 0.9078
Epoch 3/50
500/500 [==============================] - 343s 685ms/step - loss: 0.1308 - acc: 0.9553 - f1_m: 0.9551 - precision_m: 0.9580 - recall_m: 0.9524 - val_loss: 0.3038 - val_acc: 0.9183 - val_f1_m: 0.9182 - val_precision_m: 0.9207 - val_recall_m: 0.9158
Epoch 4/50
500/500 [==============================] - 343s 685ms/step - loss: 0.5172 - acc: 0.8705 - f1_m: 0.8693 - precision_m: 0.8801 - recall_m: 0.8592 - val_loss: 0.3254 - val_acc: 0.8960 - val_f1_m: 0.8963 - val_pre

In [ ]:
# Loss and accuracy curves
# summarize_diagnostics(history_3)

In [22]:
# # Evaluate model, wont work as we have no test data.
# _, test_acc, test_auc = model_3.evaluate(test_ds, verbose=1)
# print('AUC:', test_auc)
# print('Accuracy:', test_acc)

# Evaluate model, wont work as we have no test data.
# _, test_acc, test_auc = model.evaluate(test_ds, verbose=1)
loss, accuracy, f1_score, precision, recall = model_3.evaluate(test_ds, verbose=1)
# print('AUC:', test_auc)
# print('Accuracy:', test_acc)
print('loss:', loss)
print('Accuracy:', accuracy)
print('f1_score:', f1_score)
print('precision:', precision)
print('recall:', recall)

125/125 [==============================] - 18s 143ms/step - loss: 0.3657 - acc: 0.9025 - f1_m: 0.9027 - precision_m: 0.9036 - recall_m: 0.9018
loss: 0.3656522333621979
Accuracy: 0.9024999737739563
f1_score: 0.9026506543159485
precision: 0.9035806059837341
recall: 0.9017500281333923
